# Denoising Methods - SVD

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pywt
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import plotly.graph_objects as go

In [ ]:
# Noisy
# Example images
img1_file = 'data/sidd/samples/sample_sidd_validation_NOISY__1.png'
img2_file = 'data/sidd/samples/sample_sidd_validation_NOISY__2.png'
img3_file = 'data/sidd/samples/sample_sidd_validation_NOISY__3.png'
img4_file = 'data/sidd/samples/sample_sidd_validation_NOISY__4.png'

# Load image using cv2
img1 = cv2.imread(img1_file)
img2 = cv2.imread(img2_file)
img3 = cv2.imread(img3_file)
img4 = cv2.imread(img4_file)

# Convert BGR image to RGB
img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img3_rgb = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
img4_rgb = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)

# GT
# Example images
img1_file_gt = 'data/sidd/samples/sample_sidd_validation_GT__1.png'
img2_file_gt = 'data/sidd/samples/sample_sidd_validation_GT__2.png'
img3_file_gt = 'data/sidd/samples/sample_sidd_validation_GT__3.png'
img4_file_gt = 'data/sidd/samples/sample_sidd_validation_GT__4.png'

# Load image using cv2
img1_gt = cv2.imread(img1_file_gt)
img2_gt = cv2.imread(img2_file_gt)
img3_gt = cv2.imread(img3_file_gt)
img4_gt = cv2.imread(img4_file_gt)

# Convert BGR image to RGB
img1_rgb_gt = cv2.cvtColor(img1_gt, cv2.COLOR_BGR2RGB)
img2_rgb_gt = cv2.cvtColor(img2_gt, cv2.COLOR_BGR2RGB)
img3_rgb_gt = cv2.cvtColor(img3_gt, cv2.COLOR_BGR2RGB)
img4_rgb_gt = cv2.cvtColor(img4_gt, cv2.COLOR_BGR2RGB)

In [ ]:
# Create subplots
fig = make_subplots(rows=1, cols=3, subplot_titles=("Red Channel", "Green Channel", "Blue Channel"))
|
fig.add_trace(go.Heatmap(z=img1_rgb_gt[:, :, 0], colorscale='greys_r'), row=1, col=1)  # Red Channel
fig.add_trace(go.Heatmap(z=img1_rgb_gt[:, :, 1], colorscale='greys_r'), row=1, col=2)  # Green Channel
fig.add_trace(go.Heatmap(z=img1_rgb_gt[:, :, 2], colorscale='greys_r'), row=1, col=3)  # Blue Channel

# Update layout
fig.update_layout(height=400, width=900, title_text="Color Channels of the Image")

# Show the plot
fig.show()

## Initial Image SVD

In [ ]:
U_1, s_1, Vt_1 = np.linalg.svd(img1_rgb_gt[:, :, 0] / 255, full_matrices=False)
U_2, s_2, Vt_2 = np.linalg.svd(img1_rgb_gt[:, :, 1] / 255, full_matrices=False)
U_3, s_3, Vt_3 = np.linalg.svd(img1_rgb_gt[:, :, 2] / 255, full_matrices=False)

In [ ]:
fig = px.scatter(y=s_1, title='Singular Values')
fig.add_trace(go.Scatter(y=s_2, mode='markers'))
fig.add_trace(go.Scatter(y=s_3, mode='markers'))

fig.update_layout(showlegend=False)

# Show the plot
fig.show()

In [ ]:
rank = 10

c1 = np.dot(U_1[:, :rank], np.dot(np.diag(s_1[:rank]), Vt_1[:rank, :]))
c2 = np.dot(U_2[:, :rank], np.dot(np.diag(s_2[:rank]), Vt_2[:rank, :]))
c3 = np.dot(U_3[:, :rank], np.dot(np.diag(s_3[:rank]), Vt_3[:rank, :]))

c1 = np.clip(c1 * 255, 0, 255).astype(np.uint8)
c2 = np.clip(c2 * 255, 0, 255).astype(np.uint8)
c3 = np.clip(c3 * 255, 0, 255).astype(np.uint8)


In [ ]:
# Create subplots
fig = make_subplots(rows=1, cols=3, subplot_titles=("Red Channel", "Green Channel", "Blue Channel"))

fig.add_trace(go.Heatmap(z=c1, colorscale='greys_r'), row=1, col=1)  # Red Channel
fig.add_trace(go.Heatmap(z=c2, colorscale='greys_r'), row=1, col=2)  # Green Channel
fig.add_trace(go.Heatmap(z=c3, colorscale='greys_r'), row=1, col=3)  # Blue Channel

# Update layout
fig.update_layout(height=400, width=900, title_text="Color Channels of the Reconstructed Image with Rank " + str(rank))

# Show the plot
fig.show()

In [ ]:
fig = px.imshow(cv2.merge([c1, c2, c3]))
fig.show()

In [ ]:
def denoise_image_svd(image, rank):
    # Split the image into color channels
    channels = cv2.split(image)
    denoised_channels = []
    
    for channel in channels:
        # Perform SVD
        U, s, Vt = np.linalg.svd(channel / 255, full_matrices=False)
        
        # Reconstruct the image using only the top 'rank' singular values
        denoised_channel = np.dot(U[:, :rank], np.dot(np.diag(s[:rank]), Vt[:rank, :]))
        
        # Return Image to [0, 255] - Clip values to be between 0 and 255
        denoised_channel = np.clip(denoised_channel * 255, 0, 255).astype(np.uint8)
        denoised_channels.append(denoised_channel)
    
    # Merge the denoised channels
    denoised_image = cv2.merge(denoised_channels)
    return denoised_image

In [ ]:
ranks = list(range(1, 100))

metrics = [0 for i in ranks]
best_l1 = 1e4
best_rank = -1
best_index = -1

for i, rank in enumerate(ranks):
    img1_rgb_svd = denoise_image_svd(img1_rgb, rank)
    img2_rgb_svd = denoise_image_svd(img2_rgb, rank)
    img3_rgb_svd = denoise_image_svd(img3_rgb, rank)
    img4_rgb_svd = denoise_image_svd(img4_rgb, rank)

    l1_1 = abs(img1_rgb_gt - img1_rgb_svd).mean()
    l1_2 = abs(img2_rgb_gt - img2_rgb_svd).mean()
    l1_3 = abs(img3_rgb_gt - img3_rgb_svd).mean()
    l1_4 = abs(img4_rgb_gt - img4_rgb_svd).mean()

    l1_avg = (l1_1 + l1_2 + l1_3 + l1_4) / 4
    metrics[i] = l1_avg

    if l1_avg < best_l1:
        best_rank = rank
        best_l1 = l1_avg
        best_index = i


In [ ]:
# Create a Plotly Express scatter plot
fig = px.scatter(x=ranks, y=metrics, title='Plot of Array Values', labels={'x':'rank', 'y':'Loss'})

# Add a marker for the minimum point
fig.add_trace(go.Scatter(
    x=[best_rank],
    y=[metrics[best_index]],
    mode='markers+text',
    marker=dict(size=5, color='red', symbol='star'),
    text=['MIN'],
    textposition="top center"
))

fig. update_layout(showlegend=False)

# Show the plot
fig.show()


In [ ]:
best_rank = 2

In [ ]:
# Denoise using wavelet transform
img1_rgb_svd = denoise_image_svd(img1_rgb, best_rank)
img2_rgb_svd = denoise_image_svd(img2_rgb, best_rank)
img3_rgb_svd = denoise_image_svd(img3_rgb, best_rank)
img4_rgb_svd = denoise_image_svd(img4_rgb, best_rank)

# Create subplots for 4x3 layout
fig = make_subplots(rows=4, cols=3, subplot_titles=(
    "Original Image 1", "Denoised Image 1", "Ground Truth 1",
    "Original Image 2", "Denoised Image 2", "Ground Truth 2",
    "Original Image 3", "Denoised Image 3", "Ground Truth 3",
    "Original Image 4", "Denoised Image 4", "Ground Truth 4"
))

fig.add_trace(go.Image(z=img1_rgb), row=1, col=1)  # Original Image 1
fig.add_trace(go.Image(z=img1_rgb_svd), row=1, col=2)  # Denoised Image 1
fig.add_trace(go.Image(z=img1_rgb_gt), row=1, col=3)  # Ground Truth 1

fig.add_trace(go.Image(z=img2_rgb), row=2, col=1)  # Original Image 2
fig.add_trace(go.Image(z=img2_rgb_svd), row=2, col=2)  # Denoised Image 2
fig.add_trace(go.Image(z=img2_rgb_gt), row=2, col=3)  # Ground Truth 2

fig.add_trace(go.Image(z=img3_rgb), row=3, col=1)  # Original Image 3
fig.add_trace(go.Image(z=img3_rgb_svd), row=3, col=2)  # Denoised Image 3
fig.add_trace(go.Image(z=img3_rgb_gt), row=3, col=3)  # Ground Truth 3

fig.add_trace(go.Image(z=img4_rgb), row=4, col=1)  # Original Image 4
fig.add_trace(go.Image(z=img4_rgb_svd), row=4, col=2)  # Denoised Image 4
fig.add_trace(go.Image(z=img4_rgb_gt), row=4, col=3)  # Ground Truth 4

# Remove axes for all subplots
fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)

fig.update_layout(height=1000, 
                  width=900, 
                  title_text="Image Denoising using SVD (4x3 Layout)",
                  margin=dict(l=20, r=20, t=80, b=20),  # Reduce margins
)

fig.show()